In [1]:
import os
os.chdir("C:\\Users\\erin.melly\\Desktop\\GitHub\\Tax-Calculator")
import behresp
import microdf as mdf
from taxcalc import *
import pandas as pd
import numpy as np
import copy

In [2]:
# read in PUF
puf_df = pd.read_csv(r'C:\Users\erin.melly\Desktop\GitHub\Tax-Calculator\PUF.csv')

# set year
cyr = 2022

# elasticity dictionary 
cg= {2021: -4.184,
     2022: -3.488,
     2023: -2.76,
     2024: -2.76,
     2025: -2.76,
     2026: -2.76,
     2027: -2.76,
     2028: -2.76,
     2029: -2.76,
     2030: -2.76
    }
# capital gains realization at death dictionary
realization_at_death = {2021: 204253570343, 
                        2022: 215627184994,
                        2023: 226688939254, 
                        2024: 235220544654,
                        2025: 243736672365,
                        2026: 245533163473,
                        2027: 254865207631,
                        2028: 264380671657,
                        2029: 274178230759,
                        2030: 284077457991
                       }

In [3]:
# create new variable for gains taxed at death in PUF

# share of capital gains
puf_df = puf_df.assign(ltgains_weight = (puf_df['p23250'] * puf_df['s006']) / sum(puf_df['p23250'] * puf_df['s006']))
puf_df.loc[(puf_df['p23250'] > 0), 'ltgains_weight'] = (puf_df.loc[(puf_df['p23250'] > 0), 'p23250'] * puf_df.loc[(puf_df['p23250'] > 0), 's006']) \
/ sum(puf_df.loc[(puf_df['p23250'] > 0), 'p23250'] * puf_df.loc[(puf_df['p23250'] > 0), 's006'])
# assign total realization at death revenue from JCT to taxpayers based on capital gains weight
puf_df['gains_at_death'] = (puf_df['ltgains_weight'] * realization_at_death[cyr]) / (puf_df['s006'] / 100) 

In [4]:
# current law calculator
rec = Records(data= puf_df)
pol = Policy()
calc= Calculator(pol, rec)
calc.advance_to_year(cyr)
calc.calc_all()
df = calc.dataframe([], all_vars=True)
rev1static = calc.weighted_total('combined')

In [5]:
# biden reform dictionary
biden = {
    #restore top brackets
    "II_brk5": {2020: [416700.00, 416700.00, 208350.00, 416700.00, 416700.00]},
    "II_rt5": {2020: 0.33},
    "II_brk6": {2020: [418400.00, 470700.00, 235350.00, 444550.00, 470700.00]},
    "II_rt7": {2020: 0.396},
    "PT_brk5": {2020: [416700.00, 416700.00, 208350.00, 416700.00, 416700.00]},
    "PT_rt5": {2020: 0.33},
    "PT_brk6": {2020: [418400.00, 470700.00, 235350.00, 444550.00, 470700.00]},
    "PT_rt7": {2020: 0.396},
    # restore Pease limitations
    "ID_ps": {2020: [261500.0, 313800.0, 156900.0, 287650.0, 313800.0]},
    "ID_prt": {2020: 0.03},
    "ID_crt": {2020: 0.8},
    # Phase out section 199A for high-income
    "PT_qbid_taxinc_thd": {2020: [400000.00, 400000.00, 400000.00, 400000.00, 400000.00]},
    "PT_qbid_taxinc_gap": {2020: [100000.0, 100000.0, 100000.0, 100000.0, 100000.00]},
    # tax capital gains as ordinary income above $10 million
    "CG_brk4": {2020: [1000000, 1000000, 1000000, 1000000, 1000000]},
    "CG_rt3": {2020: 0.396},
    # capital gains realization at death
    "CG_TaxAtDeath": {2020: True},
    "CG_exclusion_thd": {2020: [100000, 200000, 100000, 100000, 200000]},
    # extend childless EITC to workers above 65 years old
    "EITC_MaxEligAge": {2020: 125},
    # limit the value of ID to 28% of value
    #"ID_Biden_Limit": {2020: True},
    "ID_BenefitSurtax_trt": {2020: 1.0},
    "ID_BenefitSurtax_crt": {2020: 0.28},
    "ID_BenefitSurtax_Switch": {2020: [True, True, False, True, True, True, True]},
    # apply earnings above $400K to OASDI taxHan
    "SS_Earnings_thd": {2020: 400000},
    # replace deductibility of IRA/DC pension contributions with 26% credit
    "ALD_IRAContributions_hc": {2020: 1.0},
    "ALD_KEOGH_SEP_hc": {2020: 1.0},
    "IRADC_credit_rt": {2020: 0.26},
    # reinstate energy efficiency credit
    "CR_ResidentialEnergy_hc": {2020: 1.0}
}

In [ ]:
# Revenue Change from Reform, with Behavioral Responses

# specify Calculator object for static analysis of reform policy
pol.implement_reform(biden)
calc2 = Calculator(pol, rec)

# calculate reform income tax liabilities for cyr under static assumptions
calc2.advance_to_year(cyr)
calc2.calc_all()
rev2static = calc2.weighted_total('combined')

# specify assumed non-zero response-function substitution elasticity
response_elasticities = {'sub': 0.25,
                         'cg': cg[cyr]}

# specify Calculator object for analysis of reform with behavioral responses
calc2 = Calculator(policy=pol, records=rec)
calc2.advance_to_year(cyr)
_, df2br = behresp.response(calc, calc2, response_elasticities)

# calculate reform income tax liabilities for cyr with behavioral response
rev2br = (df2br['combined'] * df2br['s006']).sum()

# print total income tax revenue estimates for cyr (estimates in billons of dollars)
print('{}_DYNAMIC_DIFERENCE_tot_rev($B)= {:.3f}'.format(cyr, (rev2br - rev1static) * 1e-9))

In [ ]:
## Static Distributional Analysis

In [6]:
# specify Calculator object for static analysis of reform policy
pol.implement_reform(biden)
calc2 = Calculator(pol, rec)
calc2.advance_to_year(cyr)
calc2.calc_all()

In [7]:
calc.difference_table(calc2, groupby='standard_income_bins', tax_to_diff='combined') 

,count,tax_cut,perc_cut,tax_inc,perc_inc,mean,tot_change,share_of_change,ubi,benefit_cost_total,benefit_value_total,pc_aftertaxinc
<$0K,1.33,0.01,0.8,1.04,78.2,14262.1,18.912,4.9,0.0,0.0,0.0,10.3
=$0K,3.54,0.00,0.0,0.10,3.0,12.6,0.044,0.0,0.0,0.0,0.0,-8.9
$0-10K,19.46,0.09,0.5,0.17,0.9,12.7,0.246,0.1,0.0,0.0,0.0,-0.3
$10-20K,22.29,0.38,1.7,0.14,0.6,7.0,0.155,0.0,0.0,0.0,0.0,-0.0
$20-30K,19.08,0.67,3.5,0.15,0.8,-2.5,-0.048,-0.0,0.0,0.0,0.0,0.0
$30-40K,16.50,0.58,3.5,0.17,1.0,-3.8,-0.063,-0.0,0.0,0.0,0.0,0.0
$40-50K,12.84,0.43,3.4,0.17,1.3,-3.7,-0.047,-0.0,0.0,0.0,0.0,0.0
$50-75K,25.81,0.56,2.2,0.62,2.4,4.5,0.117,0.0,0.0,0.0,0.0,-0.0
$75-100K,16.25,0.43,2.7,0.61,3.7,4.5,0.073,0.0,0.0,0.0,0.0,-0.0
$100-200K,29.07,1.03,3.5,1.66,5.7,13.1,0.381,0.1,0.0,0.0,0.0,-0.0
